Bienvenue dans ce premier TP d’apprentissage automatique !

En tant qu’étudiant.e.s, la question “Combien d’heures devrais je réviser pour obtenir une bonne note à mon cours d’IA et apprentissage?” peut rapidement nous tarauder, si ce n’est être tout simplement source d’anxiété.

Aujourd’hui, nous utiliserons l’apprentissage automatique pour tenter de répondre à cette question et tous nous rassurer. Pour ce faire, nous allons développer un modèle de régression linéaire univariée qui aura pour but de prédire la note qu'un.e étudiant.e peut espérer obtenir en fonction du nombre d'heures qu’iel aura consacrées à la révision.

Comme nous l’avons vu précédemment, la première étape nécessaire à l’élaboration de pareil modèle consiste en la récolte et création d’un jeu de données. Pour ce faire, nous avons recueilli l’année dernière des données sur 70 étudiants de la L3 Informatique et vidéoludique de Paris 8. Chaque étudiant a été invité à fournir deux informations cruciales :
Le nombre d'heures qu'iel a consacrées à la révision pour le partiel.
La note qu'iel a obtenue à ce partiel.
Nous avons minutieusement enregistré ces informations et avons créé un ensemble de données représentatif pour notre analyse.



**Objectif du TP**

Votre mission pour ce TP est de développer un modèle de régression linéaire univariée qui permettra de prédire la note potentielle d'un.e étudiant.e en fonction du temps de révision qu'iel consacrera pour le partiel. Vous allez apprendre à utiliser ces données pour construire le modèle, le former et évaluer ses performances.

Commencez par executez la case suivante, qui importe toutes les librairies nécessaire à l'execution du TP.

In [ ]:
import torch
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Certaines de ces librairies vous seront peut-être familières:

*   Numpy: Librairie de calcul vectoriel
*   Pandas: Librairie de manipulation de tables
*   Matplotlib/Pyplot: Librairie de visualisation de données

Certaines, en revanches, sont potentiellement nouvelles pour vous:
*  PyTorch (torch): Une des librairies de deep learning les plus usitées dans le monde actuellement.
*  Scikit-learn (sklearn): Librairie standard de machine learning classique




#  1. Chargement et visualisation des données

Maintenant que notre environnement de travail est prêt, nous allons pouvoir charger notre jeu de données et regarder un peu à quoi il ressemble. Uploadez le fichier "train.csv" fourni avec ce notebook dans vos fichiers, et chargez le dans le jupyter avec la fonction pandas pd.read_csv

In [ ]:
train_df = ### CODEZ ICI ###

Affichez une description de votre jeu de données ici en utilisant la méthode .describe() associée à votre dataframe pandas

In [ ]:
### CODEZ ICI ###

En utilisant la librairie pyplot, visualisez la distribution des variables prédictives et à expliquer de votre jeu de données, ainsi que leur relation.

Vous pourrez pour ce faire utiliser les fonctions pyplot plt.hist et plt.scatter

In [ ]:
### CODEZ ICI ###

Décrivez ici qualitativement votre jeu de données.

Un modèle de régression linéaire semble t'il indiqué ici pour implémenter un algorithme capable de déterminer la note qu'un étudiant aura au partiel à partir de son temps de révision?

(Ecrivez votre réponse dans cette case)

# 2. Création et ajustement d'un modèle de régression linéaire avec Pytorch.

Maintenant que l'on connait mieux notre jeu de données, il est grand temps d'implémenter et d'ajuster notre modèle. Nous allons pour cela utiliser PyTorch, une des librairies actuellement les plus utilisées pour implémenter des modèles d'apprentissage profond (mais qui permet tout autant d'ajuster des modèles linéaires).

# 2.1. Premiers pas avec Torch et implémentation du modèle

Avant d'implémenter notre modèle il nous sera utile de nous familiariser avec la façon dont on effectue des calculs avec PyTorch.

Pytorch encode les entrées, la sortie et les paramètres d'un modèle dans un objet appelé Tensor. Un Tensor peut être assimilé conceptuellement à un array Numpy, mais dispose de méthodes supplémentaires (permettant notamment à PyTorch de calculer automatiquement le gradient de notre fonction objectif).

On peut convertir un array numpy (ou une série pandas) simplement en appelant torch.tensor() sur l'objet que l'on souhaite convertir en Tensor.

On peut de la même manière convertir un Tensor en numpy array en appelant sa méthode .numpy()


In [ ]:
# Convertit le nombre d'heures étudiées de chaque étudiant en Tensor
X = torch.tensor(train_df.num_heure)
y = torch.tensor(train_df.note)

print("A quoi ressemble un Tensor?")
print(X)

print("A quoi ressemble le même Tensor transformé en array numpy?")
print(X.numpy())

PyTorch dispose également d'un autre objet pour encoder des données, appelé Parameter (torch.nn.Parameter).

Comme son nom l'indique, cet objet sera utilisé pour définir les paramètres de notre modèle. Cet objet a globalement le même comportement qu'un Tensor, à la différence près que PyTorch peut reconnaitre automatiquement les instances de cet objet comme des paramètres, et les modifier pendant la descente de gradient.

On peut instancier directement un objet Parameter à partir d'une instance de Tensor en appelant simplement torch.nn.Parameter(un objet Tensor)

In [ ]:
print("A quoi ressemble un objet Parameter?")
print(torch.nn.Parameter(X))

On constate en executant la cellule précédente qu'une instance de Parameter dispose d'un attribut booléen "requires_grad". Cet attribut permet de signaler à PyTorch si cet paramètre doit être ajusté pendant la descent de gradient.

Maintenant que nous sommes un peu plus familier avec les objets que PyTorch utilise pour manipuler des données, nous pouvons passer à l'implémentation du modèle.

L'implémentation d'un modèle en pytorch se fait systématiquement par l'implémentation d'un objet héritant d'un objet de pytorch, un Module (torch.nn.Module).

Implémenter notre modèle revient donc à créer un objet héritant de Module dont on implémentera deux méthodes:

*   La méthode __init__ où l'on définiera les paramètres de notre modèle en tant qu'attributs
*   Une méthode appelée forward, qui admet un argument, input (un tensor Torch) et qui retourne également un Tensor. C'est dans cette méthode que nous implémentererons la logique de notre couche, c'est à dire la transformation que l'on désire que notre modèle applique au Tensor input

Une fois instancié, un objet Module se comporte comme une fonction python, qu'on peut appeler avec un Tensor, et qui retourne en sortie le Tensor résultant de la transformation que nous avons défini dans la méthode forward.

Pour rappel, un modèle de régression linéaire cherche à prédire la variable cible y à partir de la variable d'entrée x en appliquant à cette dernière la transformation suivante:

$$ y = coef * x + ordonnée $$

In [ ]:
class LinearRegression(torch.nn.Module):
    def __init__(self):
        """
        La méthode __init__ est utilisée pour définir tous les paramètres du
        modèle (les coefficients que l'on va pouvoir faire varier pendant
        la descente de gradient)
        """
        super(LinearRegression, self).__init__()
        # Définissez ici le coefficient directeur de notre modèle, comme une
        # instance de Parameter initialisé par un Tensor de valeur [1]
        self.coefficient = # CODEZ ICI #
        # Déginissez ici l'ordonnée à l'origine du modèle, comme une
        # instance de Parameter initialisé par un Tensor de valeur [0]
        self.ordonnee = # CODEZ ICI #

    def forward(self, input):
        """
        La méthode forwad est utilisée pour définir la logique de notre modèle.
        c'est à dire la transformation que l'on veut appliquer à l'entrée input
        avec nos paramètres self.coefficient et self.ordonnee
        """
        output = # CODEZ ICI #
        return output

On peut maintenant créer une instance de notre modèle de régression linéaire comme suit:

In [ ]:
model = LinearRegression()

Comme indiqué précédemment, une instance de Module se comporte comme une fonction python, à laquelle on peut fournir comme entrée nos variables prédictives comme suit:

In [ ]:
y_pred = model(X)

Calculez les performances du modèle en utilisant la fonction sklearn mean_squared_error, puis, à l'aide de la fonction plt.scatter, visualisez la relation existente entre X et y_pred.

In [ ]:
from sklearn.metrics import mean_squared_error

mse = # CODEZ ICI #
print(mse)

Que pensez vous des performances du modèle jusqu'ici?

(Ecrivez votre réponse dans cette case)

Evidemment, notre modèle n'a pas encore été ajusté. Il n'y a donc aucune raison pour qu'il soit capable de prédire correctement la note qu'un.e étudiant.e peut espérer recevoir au partiel à partir du nombre d'heure d'heure qu'iel a réviser.

Il va donc nous falloir faire implémenter une fonction qui nous permettra non seulement d'instancier un modèle, mais également de l'ajuster!

# 2.1. Ajustement du modèle

Comme vu en cours, l'implémentation d'un modèle d'apprentissage machine dépend de la définition de 4 entités distinctes:


*   Un jeu de données constitué d'exemples de paires variable prédictive - variable à prédire
*   Un modèle, définit comme une fonction paramétrique qui prend comme argument la variable prédictive et essaie de rendre en sortie une approximation de la variable à prédire
*   Une fonction objectif (ou fonction coût) qui prend comme argument les paramètre du modèle et rend en sortie un scalaire indiquant à quel point notre modèle "prédit bien" la variable à prédire à partir de la variable prédictive
*   Un algorithme d'optimisation pour trouver le minimum de cette fonction objectif (et ainsi trouver notre modèle comme l'argument de ce minimum)

Le jeu de données étant collecté et le modèle déjà implémenté, il ne nous reste plus qu'à définir notre fonction objectif, et notre algorithme d'optimisation!

PyTorch propose tout une variété d'algorithme d'optimisation et de fonction objectif, en fonction des besoins et des envies. Nous nous intérésserons dans ce TP aux deux objets suivant:


*   torch.nn.MSE comme fonction objectif (MSE pour Mean Squared Error)
*   torch.nn.SGD comme algorithme de descente de gradient (SGD pour Stochastic Gradient Descent)

Je vous invite vivement à regarder la documentation de PyTorch ([documentation de PyTorch](https://pytorch.org/docs/stable/index.html) afin de découvrir le comportement de ces objets, et comment les utiliser pour ajuster un modèle par descente de gradient.

Une fois familier avec ces deux objets, complétez l'implémentation de la fonction train_model définie dans la cellule suivante!


In [ ]:
def train_model(X, y, num_iter=10, learning_rate=0.01):

    # On instancie notre modèle
    model = # CODEZ ICI #

    # On définit notre fonction perte
    objectif = # CODEZ ICI #

    # On définit notre algorithme d'optimisation (SGD: Stochastic Gradient Descent)
    optimizer = # CODEZ ICI #

    trajectory = []

    for _ in range(num_iter):
        inputs = torch.tensor(X)
        targets = torch.tensor(y)

        # Calculez la valeur de la fonction objectif
        outputs = model(inputs)
        loss = objectif(outputs, targets)

        # Calcule le gradient
        optimizer.zero_grad()
        loss.backward()

        # Applique une itération de descente de gradient aux paramètres du modèle
        optimizer.step()

        # Conserve la valeur des paramètres du modèles pour visualiser son évolution
        trajectory.append(np.concatenate(
            [
                model.coefficient.detach().flatten().numpy(),
                model.ordonnee.detach().flatten().numpy(),
                loss.detach().flatten().numpy()
            ]
        ))

    return model, np.stack(trajectory, axis=0)

Maintenant que la fonction train_model est implémentée, nous pouvons l'appeler sur notre jeu de données et obtenir un modèle ajusté!

Pour évaluer ses performances, il nous suffit simplement par la suite d'obtenir les prédictions du modèle en lui donnant en entrée nos variables explicatives, puis de calculer l'erreur quadratique, via PyTorch ou sklearn, au choix.

Ajuster et évaluer les performances de votre modèle pour les valeurs suivantes de l'argument learning rate: [1, 0.01, 0.001]

In [ ]:
for learning_rate in [1., .1, .001]:
  predictions = # CODEZ ICI#
  mse = # CODEZ ICI #
  print("Learning_rate=%.3f : Erreur quadratique=%.3f" % (learning_rate, mse))

Que constatez vous?

(Ecrivez votre réponse dans cette cellule)

Afin de mieux comprendre ce qui peut bien se passer, je vous ai implémenté une fonction "plot_cost_and_trajectory" qui nous permettra de visualiser l'évolution du modèle que vous allez implémenté pendant son ajustement.

In [ ]:
def plot_cost_and_trajectory(X, y, learning_rate=0.01, num_points=200):

    def build_cost_function_mesh(slope_bound, intercept_bound, num_points=200):
        # Generate a meshgrid of slope and intercept values
        slope_range = np.linspace(
            slope_bound[0],
            slope_bound[1] + (slope_bound[1] - slope_bound[0]),
            num_points
        )
        intercept_range = np.linspace(
            intercept_bound[0],
            2 * intercept_bound[1] - intercept_bound[0],
            num_points
        )
        slope_values, intercept_values = np.meshgrid(slope_range, intercept_range)

        # Calculate mean square error for each combination of slope and intercept
        mse_values = np.zeros_like(slope_values)
        for i in range(num_points):
            for j in range(num_points):
                slope = slope_values[i, j]
                intercept = intercept_values[i, j]
                y_pred = slope * X + intercept
                mse_values[i, j] = np.mean((y_pred - y) ** 2)

        return slope_values, intercept_values, mse_values

    trajectory = train_model(X, y, learning_rate=learning_rate)

    trajectory = np.concatenate([trajectory, np.arange(len(trajectory)).reshape(-1, 1)], axis=-1)

    slope_values, intercept_values, mse_values = build_cost_function_mesh(
        slope_bound=(trajectory[:, 0].min(), trajectory[:, 0].max()),
        intercept_bound=(trajectory[:, 1].min(), trajectory[:, 1].max()),
    )

    # Plot the 3D surface
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(slope_values, intercept_values, mse_values, cmap='viridis', alpha=0.8)

    sc = ax.scatter(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], c=trajectory[:, 3], cmap='plasma', marker='o',
                    label='Gradient Descent')

    # Plot lines connecting consecutive dots
    for i in range(len(trajectory) - 1):
        ax.plot([trajectory[i, 0], trajectory[i + 1, 0]],
                [trajectory[i, 1], trajectory[i + 1, 1]],
                [trajectory[i, 2], trajectory[i + 1, 2]], color='blue', linestyle='dashed')

    ax.set_xlabel('Coefficient directeur')
    ax.set_ylabel("Ordonnée à l'origine")
    ax.set_zlabel('Erreur quadratique')
    ax.set_title("Visualisation de l'évolution du modèle pour learning_rate=%.3f" % learning_rate)
    plt.show()

    return None


Executez les trois prochaines cellules pour visualiser l'évolution du modèle au cours de la descent de gradient pour les différentes valeurs de learning_rate.


In [ ]:
plot_cost_and_trajectory(X, y, learning_rate=1.)

In [ ]:
plot_cost_and_trajectory(X, y, learning_rate=.1)

In [ ]:
plot_cost_and_trajectory(X, y, learning_rate=.01)

Que constatez vous?

(Ecrivez votre réponse dans cette cellule)

Cette petite expérience nous montre clairement l'importance du choix de valeur pour la variable learning_rate. Trop faible, et l'algorithme de descent approche trop lentement du minimum de la fonction objectifr. Trop élevée, et la descente diverge tout simplement, jusqu'à atteindre des valeurs de fonction objectif stratosphériques.

# Implémentation du modèle avec scikit-learn

Félicitation! Vous venez d'implémenter votre premier modèle de machine learning avec Pytorch. Comme vous avez pu le constater par vous même, il est nécessaire pour y parvenir de coordonner pas mal d'objet un peu capricieux entre eux.

En revanche, une fois que vous aurez parfaitement intégré ce mode de fonctionnement, implémentez un modèle d'apprentissage profond moderne vous semblera un jeu d'enfant!

Pour ce qui est d'un modèle de régression linéaire en revanche, c'est un peu se complexifier la vie pour rien. En effet, on peut parvenir au même résultats en quelques lignes de codes avec la librairie scikit-learn, la librairie la plus connue pour faire du machine learning dit "classique" (c'est à dire "non profond" en gros).

Voyez plutôt:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = X.numpy().reshape([-1, 1])
y = y.numpy().reshape([-1, 1])

model = LinearRegression()
model.fit(X, y)
y_preds = model.predict(X)
mse = mean_squared_error(y_preds, y)
print("Erreur quadratique du modèle: %.3f" % mse)

Clairement, il est bien plus pratique d'utiliser scikit learn pour ajuster des modèles simples comme les modèles de régression linéaire.

En revanche, notre implémentation avec pyTorch nous a permis de vraiment comprendre toute l'implémentation nécessaire à l'ajustement d'un modèle de régression. De plus, la complexité additionnelle de l'implémentation avec PyTorch devient absolument nécessaire en apprentissage profond. Donc autant se familiariser avec dès le début!

Maintenant que nous sommes capables d'ajuster des modèles très simplement avec scikit-learn, nous pouvons en revanche faire plein d'expériences!

On pourrait par exemple se poser demander comment impacte la taille de notre jeu de données sur les performances du modèle.

Pour ce faire, ajustez un modèle (que vous appelerez model_2) sur les 15 premières observations du jeu de données, puis un autre (que vous appelerez model_3) sur les 5 premières observations du jeu de données.

Une fois les deux modèles ajustés, évaluez leurs performances (leur erreur quadratique) deux fois:
*   Sur les observations que vous avez utilisé pour ajuster les deux modèles (donc les 15 premières pour model_2 et les 5 premières pour model_3)
*   Sur les observations que vous n'avez pas utilisé pour ajuster les deux modèles (donc les 55 dernières pour model_2 et les 65 dernières pour model_3)

In [ ]:
# CODEZ ICI #

Que constatez vous?

(Ecrivez votre réponse dans cette cellule)